### 1.Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import flask

### 2. Import (cleaned) data an select 7 Attributs

In [2]:
df = pd.read_csv('../model/Final_Documents/cleaned_dataframe.csv')
df = df[['AreaLiving','AreaProperty','BuiltYear','Rooms','Zip','RealEstateTypeId','Renovationyear','PurchasePrice']]
df = df.fillna(0)
df.loc[df['Renovationyear'] > 0, 'Renovationyear'] = 1
df

,AreaLiving,AreaProperty,BuiltYear,Rooms,Zip,RealEstateTypeId,Renovationyear,PurchasePrice
0,140.0,501.0,2016,4.5,4616,200,0.0,745000.0
1,143.0,277.0,2004,5.5,5033,200,0.0,780000.0
2,160.0,712.0,1945,7.0,2854,1000,1.0,570000.0
3,351.0,496.0,2016,5.5,1187,1000,0.0,920000.0
4,400.0,1800.0,1975,8.0,1292,1000,1.0,3950000.0
...,...,...,...,...,...,...,...,...
153619,116.0,0.0,2005,5.0,1227,800,0.0,1700000.0
153620,70.0,0.0,1978,4.0,1226,100,0.0,720000.0
153621,136.0,0.0,1997,5.5,1116,100,0.0,725000.0
153622,60.0,0.0,2009,2.5,1996,100,0.0,289000.0


### 3. train Model

In [3]:
X, y = df.drop(['PurchasePrice'], axis=1), df['PurchasePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
model=xgb.XGBRegressor(n_estimators = 100,eta = 0.05, max_depth = 15,alpha = 0.2, colsample_bytree = 0.4)
model.fit(X_train, np.log(y_train))
y_pred = model.predict(X_test)
y_pred = np.exp(y_pred)
np.mean(np.abs((y_test - y_pred) / y_test)) * 100

19.261930598020914

### 4. Flask GUI

In [5]:
# Initialise the Flask app
app = flask.Flask(__name__, template_folder='templates')

# Set up main route
@app.route('/', methods=['GET', 'POST'])
def main():
    if flask.request.method == 'GET':
        return(flask.render_template('main.html'))
    
    if flask.request.method == 'POST':
        # Extract the input
        Wohnflaeche = flask.request.form['AreaLiving']
        Grundstueck = flask.request.form['AreaProperty']
        Baujahr = flask.request.form['BuiltYear']
        Zimmer = flask.request.form['Rooms']
        Postleitzahl = flask.request.form['Zip']
        Gebaeudeart = flask.request.form['RealEstateTypeId']
        Renoviert = flask.request.form['Renovationyear']
        if Gebaeudeart == 'Haus':
            Gebaeudeart = 200
        elif Gebaeudeart =='Reihenhaus':
            Gebaeudeart = 400
        elif Gebaeudeart =='Doppelhaus':
            Gebaeudeart = 900
        elif Gebaeudeart =='Mehrfamilienhaus':
            Gebaeudeart = 2000
        elif Gebaeudeart =='Terassenhaus':
            Gebaeudeart = 500
        elif Gebaeudeart =='AnderesHaus':
            Gebaeudeart = 7100
        elif Gebaeudeart =='Wohnung':
            Gebaeudeart = 100
        elif Gebaeudeart =='Attikawohnung':
            Gebaeudeart = 800
        elif Gebaeudeart =='Maisonettewohnung':
            Gebaeudeart = 600
        elif Gebaeudeart =='Loftwohnung':
            Gebaeudeart = 1500
        elif Gebaeudeart =='Zimmer':
            Gebaeudeart = 4900
        
        if Renoviert == 'Ja':
            Renoviert = 1
        elif Renoviert =='Nein':
            Renoviert = 0

        # Make df for model
        input_variables = pd.DataFrame([[Wohnflaeche, Grundstueck, Baujahr,Gebaeudeart,Renoviert,Zimmer,Postleitzahl]],
                                       columns=['AreaLiving', 'AreaProperty', 'BuiltYear','Rooms','Zip','RealEstateTypeId','Renovationyear'],
                                       dtype=float,
                                       index=['input'])

        # Get the model's prediction
        prediction = model.predict(input_variables)[0]
        prediction = np.exp(prediction)
    
        # Render
        return flask.render_template('main.html',
                                     original_input={'AreaLiving':Wohnflaeche,
                                                     'AreaProperty':Grundstueck,
                                                     'BuiltYear':Baujahr,
                                                    'Rooms':Zimmer,
                                                    'Zip':Postleitzahl,
                                                    'RealEstateTypeId':Gebaeudeart,
                                                    'Renovationyear':Renoviert,},
                                     result=prediction,
                                     )

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2020 11:54:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2020 11:54:50] "POST / HTTP/1.1" 200 -
